# *Conexión al Drive*

In [1]:
# Crear conexion a drive

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
#
import os
import sys
import shutil


path='/content/drive/MyDrive/cod/marketing/MarketingprojectG8/'
os.chdir(path)###volver la carpeta del repositorio directorio de trabajo
sys.path.append(path)### agregarla al path, para poder leer los archivos de funciones propios como paquetes

# Ruta de la base de datos copiada (new database file path)

copied_path = '/content/drive/MyDrive/cod/marketing/MarketingprojectG8/movies2'


# Construcción la ruta completa al archivo de base de datos de origen
source_file = os.path.join(path, 'db_movies')

# Crear una copia de la base de datos
shutil.copy(source_file, copied_path)

# Verificar que se haya copiado la base de datos
print(f"Database copied from '{source_file}' to '{copied_path}'")


Database copied from '/content/drive/MyDrive/cod/marketing/MarketingprojectG8/db_movies' to '/content/drive/MyDrive/cod/marketing/MarketingprojectG8/movies2'


In [3]:
print(os.listdir())

['db_movies', '.git', '.gitignore.txt', 'a_funciones.py', 'README.md', 'preprocesamiento.sql', 'b_exploracion_limpieza.ipynb', 'movies2']


# *Librerias*

In [4]:
###paquete para crear y trabajar con bases de datos.
import sqlite3 as sql
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go ### para gráficos
import plotly.express as px
import a_funciones as fn
from mlxtend.preprocessing import TransactionEncoder
import re

# *Conexión a SQL*

In [5]:
# Conectar a la nueva base de datos copiada
con = sql.connect(copied_path)
cur= con.cursor()

# Consultar las tablas para verificar que la copia se realizó correctamente
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()

print("Tablas en la base de datos copiada:")
for table in tables:
    print(table)

Tablas en la base de datos copiada:
('ratings',)
('movies',)
